# Advanced Topics

Note: The following content was inspired by https://www.dataquest.io/blog/advanced-jupyter-notebooks-tutorial

## Shell Commands

Every user will benefit at least from time-to-time from the ability to interact directly with the operating system from within their notebook. Any line in a code cell that you begin with `!` will be executed as a shell command.

This can be useful when dealing with datasets or other files, and managing your Python packages. As a simple illustration:

In [ ]:
!echo Hello World!!

In [ ]:
!pip freeze | grep pandas

It is also possible to use Python variables in your shell commands by prepending a `$` symbol consistent with bash style variable names.

In [ ]:
message = 'This is nifty'
!echo $message

Note that the shell in which `!` commands are executed is discarded after execution completes, so commands like `cd` will have no effect. However, IPython magics offer a solution.

## Basic Magics

Magics are handy commands built into the IPython kernel that make it easier to perform particular tasks. Although they often resemble unix commands, under the hood they are all implemented in Python. There exists far more magics than it would make sense to cover here, but it’s worth highlighting a variety of examples. We will start with a few basics before moving on to more interesting cases.

There are two categories of magic: line magics and cell magics. Respectively, they act on a single line or can be spread across multiple lines or entire cells. To see the available magics, you can do the following:

In [ ]:
%lsmagic

As you can see, there are tons of commands! Most are listed in the official documentation, which is intended as a reference but can be somewhat obtuse in places. Line magics start with a percent character `%`, and cell magics start with two, `%%`.

It’s worth noting that `!` is really just a fancy magic syntax for shell commands, and as you may have noticed IPython provides magics in place of those shell commands that alter the state of the shell and are thus lost by `!`. Examples include `%cd`, `%alias` and `%env`.

Let’s go through some more examples.

## Autosaving

First up, the `%autosave` magic let’s you change how often your notebook will autosave to its checkpoint file:

In [ ]:
%autosave 60

It’s that easy!

## Displaying Matplotlib Plots

One of the most common line magics for data scientists is surely `%matplotlib`, which is of course for use with the most popular plotting libary for Python, Matplotlib.

In [ ]:
%matplotlib inline

Providing the `inline` argument instructs IPython to show Matplotlib plot images inline, within your cell outputs, enabling you to include charts inside your notebooks. Be sure to include this magic before you import Matplotlib, as it may not work if you do not; many import it at the start of their notebook, in the first code cell.

Now, let’s start looking at some more complex features.

## Debugging

The more experienced reader may have had concerns over the ultimate efficacy of Jupyter Notebooks without access to a debugger. But fear not! The IPython kernel has its own interface to the Python debugger, pdb, and several options for debugging with it in your notebooks. Executing the `%pdb` line magic will toggle on/off the automatic triggering of pdb on error across all cells in your notebook.

In [ ]:
%pdb

In [ ]:
raise NotImplementedError()

This exposes an interactive mode in which you can use the pdb commands.

Another handy debugging magic is `%debug`, which you can execute after an exception has been raised to delve back into the call stack at the time of failure.

Note: You can leave the interactive-mode by typing `quit()`

## Timing Execution

Sometimes in research, it is important to provide runtime comparisons for competing approaches. IPython provides the two timing magics `%time` and `%timeit`, which each has both line and cell modes. The former simply times either the execution of a single statement or cell, depending on whether it is used in line or cell mode.

In [ ]:
%time
n = 1000000

In [ ]:
%time
total = 0
for i in range(n):
    total += i

The notable difference of `%timeit` from `%time` is that it runs the specified code many times and computes an average. You can specify the number of runs with the `-n` option, but if nothing is passed a fitting value will be chosen based on computation time.

In [ ]:
%timeit sum(range(n))

## Executing Different Languages

In the output of `%lsmagic` above, you may have noticed a number of cell magics named after various programming, scripting or markup langauges, including **HTML**, **JavaScript**, **Ruby**, and **LaTeX**. Using these will execute the cell using the specified language. There are also extensions available for other languages such as **R**.

For example, to render HTML in your notebook:

In [ ]:
%%HTML
This is <em>really</em> neat!

Similarly, **LaTeX** is a markup language for displaying mathematical expressions, and can be used directly:

In [ ]:
%%latex
Some important equations:
$E = mc^2$
$e^{i pi} = -1$

## Configuring Logging

Did you know that Jupyter has a built-in way to prominently display custom error messages above cell output? This can be handy for ensuring that errors and warnings about things like invalid inputs or parameterisations are hard to miss for anyone who might be using your notebooks. An easy, customisable way to hook into this is via the standard Python `logging` module.

In [ ]:
import logging
logging.error('Jupyter highlights this so that it stands out')

In [ ]:
print('Something printed')
logging.error('An error')
logging.warning('A warning')

This actually works because Jupyter notebooks listen to both standard output streams, `stdout` and `stderr`, but handle each differently; `print` statements and cell output route to `stdout` and by default `logging` has been configured to stream over `stderr`.

This means we can configure `logging` to display other kinds of messages over `stderr` too.

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.info('This is some information')
logging.debug('This is a debug message')

We can customise the format of these messages like so:

In [ ]:
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(levelname)s: %(message)s')
handler.setFormatter(formatter)

# Override the default handler
logger.handlers = [handler]

# Demo it
logging.error('An error')
logging.warning('An error')
logging.info('An error')

Note that every time you run a cell that adds a new stream handler via `logger.addHandler(handler)`, you will receive an additional line of output each time for each message logged. We could place all the logging config in its own cell near the top of our notebook and leave it be or, as we have done here, brute force replace all existing handlers on the logger. We had to do that in this case anyway to remove the default handler.

It’s also easy to log to an external file, which might come in handy if you’re executing your notebooks from the command line as discussed later. Just use a `FileHandler` instead of a `StreamHandler`:

In [ ]:
handler = logging.FileHandler(filename='important_log.log', mode='a')

A final thing to note is that the logging described here is not to be confused with using the `%config` magic to change the application’s logging level via `%config Application.log_level="INFO"`, as this determines what Jupyter outputs to the terminal while it runs.

## Extensions

As it is an open source webapp, plenty of extensions have been developed for Jupyter Notebooks, and there is a long official list. Indeed, in the *Working with Databases* section below we use the **ipython-sql** extension.

Another of particular note is the bundle of extensions from **Jupyter-contrib**, which contains individual extensions for spell check, code folding and much more.

You can install and set this up from the command line like so:

In [ ]:
!pip install jupyter_contrib_nbextensions

In [ ]:
!jupyter contrib nbextension install --user

In [ ]:
!jupyter nbextension enable spellchecker/main

In [ ]:
!jupyter nbextension enable codefolding/main

This will install the `jupyter_contrib_nbextensions` package in Python, install it in Jupyter, and then enable the spell check and code folding extensions. Don’t forget to refresh any notebooks live at the time of installation to load in changes.

Note that Jupyter-contrib only works in regular Jupyter Notebooks, but there are new extensions for JupyterLab now being released on GitHub.

## Enhancing Charts with Seaborn

One of the most common exercises Jupyter Notebook users undertake is producing plots. But Matplotlib, Python’s most popular charting library, isn’t renowned for attractive results despite it’s customisability. Seaborn instantly prettifies Matplotlib plots and even adds some additional features pertinent to data science, making your reports prettier and your job easier. It’s included in the default Anaconda installation or easily installed via `pip install seaborn`.

Let’s check out an example. First, we’ll import our libraries and load some data:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
data = sns.load_dataset("tips")

Seaborn provides some built-in sample datasets for documentation, testing and learning purposes, which we will make use of here. This “tips” dataset is a pandas `DataFrame` listing some billing information from a bar or restaurant. We can see the size of the total bill, the tip, the gender of the payer, and some other attributes.

In [ ]:
data.head()

We can easily plot `total_bill` vs `tip` in Matplotlib.

In [ ]:
plt.scatter(data.total_bill, data.tip);

Plotting in Seaborn is just as easy! Simply set a style and your Matplotlib plots will automatically be transformed.

In [ ]:
sns.set(style="darkgrid")
plt.scatter(data.total_bill, data.tip);

What an improvement, and from only one import and a single extra line! Here, we used the darkgrid style, but Seaborn has a total of five built-in styles for you to play with: darkgrid, whitegrid, dark, white, and ticks.

But we don’t have to stop with styling: as Seaborn is closely integrated with pandas data structures, its own scatter plot function unlocks additional features.

In [ ]:
sns.scatterplot(x="total_bill", y="tip", data=data);

Now we get default axis labels and an improved default marker for each data point. Seaborn can also automatically group by categories within your data to add another dimension to your plots. 

Let’s change the colour of our markers based on whether the group paying the bill were smokers or not.

In [ ]:
sns.scatterplot(x="total_bill", y="tip", hue="smoker", data=data);

That’s pretty neat! In fact, we can take this much further, but there’s simply too much detail to go into here. As a taster though, let’s colour by the size of the party paying the bill while also discriminating between smokers and non-smokers.

In [ ]:
sns.scatterplot(x="total_bill", y="tip", hue="size", style="smoker", data=data);

Hopefully it’s becoming clear why Seaborn describes itself as a *“high-level interface for drawing attractive statistical graphics.”*

Indeed, it’s high-level enough to, for example, provide one-liners for plotting data with a line of best fit (determined through linear regression), whereas Matplotlib relies on you to prepare the data yourself. But if all you need is more attractive plots, it’s remarkably customisable; for example, if you aren’t happy with the default themes, you can choose from a whole array of standard color palettes or define your own.

For more ways Seaborn allows you to visualise the structure of your data and the statistical relationships within it, check out their examples.

## Macros

Like many users, you probably find yourself writing the same few tasks over and over again. Maybe there’s a bunch of packages you always need to import when starting a new notebook, a few statistics that you find yourself computing for every single dataset, or some standard charts that you’ve produced countless times?

Jupyter lets you save code snippets as executable macros for use across all your notebooks. Although executing unknown code isn’t necessarily going to be useful for anyone else trying to read or use your notebooks, it’s definitely a handy productivity boost while you’re prototyping, investigating, or just playing around.

Macros are just code, so they can contain variables that will have to be defined before execution. Let’s define one to use:

In [ ]:
name = 'Tim'

Now, to define a macro we first need some code to use:

In [ ]:
print('Hello, %s!' % name)

We use the `%macro` and `%store` magics to set up a macro that’s reusable across all our notebooks. It’s common to begin macro names with a double underscore to distinguish them from other variables, like so:

In [ ]:
%macro -q __hello_world 3`
%store __hello_world

The `%macro` magic takes a name and a cell number that we want to save to this macro (the number in the square brackets to the left of the cell; in this case 31 as in `In [31]`), and we’ve also passed `-q` to make it less verbose. `%store` actually allows us to save any variable for use in other sessions; here, we pass the name of the macro we created so we can use it again after the kernel is shut down or in other notebooks. Run without any parameters, `%store` lists your saved items.

Note: This input number may have changed, depending on your order of execution.  Be sure to change the number in the input above to the number for the `print('Hello, %s!' % name)` input.

To load the macro from the store, we just run:

In [ ]:
%store -r __hello_world

And to execute it, we merely need to run a cell that solely contains the macro name:

In [ ]:
__hello_world

Let’s modify the variable we used in the macro:

In [ ]:
name = 'Ben'

When we run the macro now, our modified value is picked up:

In [ ]:
__hello_world

This works because macros just execute the saved code in the scope of the cell; if `name` was undefined we’d get an error.

But macros are far from the only way to share code across notebooks.

## Executing External Code

Not all code belongs in a Jupyter Notebook. Indeed, while it’s entirely possible to write statistical models or even entire multi-part projects in Jupyter notebooks, this code becomes messy, difficult to maintain, and unusable by others. Jupyter’s flexibility is no substitute for writing well-structured Python modules, which are trivially imported into your notebooks.

In general, when your quick notebook project starts to get more serious and you find yourself writing code that is reusable or can be logically grouped into a Python script or module, you should do it! Aside from the fact that you can import your own modules directly in Python, Jupyter also lets you `%load` and `%run` external scripts to support better organised, larger-scale projects and reusability.

Tasks such as importing the same set of packages over and over for every project project are a perfect candidate for the `%load` magic, which will load an external script into the cell in which it’s executed.

But enough talk already, let’s look at an example!

The current directory contains a file named `imports.py`, which has the following code:

`
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
`

We can load this simply by writing a one-line code cell, like so:

`
%load imports.py
`

Executing this will replace the cell contents with the loaded file:

`
#%load imports.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
`

Now we can run the cell again to import all our modules and we’re ready to go!

The `%run` magic is similar, except it will execute the code and display any output, including **Matplotlib** plots. You can even execute entire notebooks this way, but remember that not all code truly belongs in a notebook...

Let’s check out an example of this magic;
The current directory contains a file named `triangle_hist.py` which contains the following short script:

`
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
if __name__ == '__main__':
    h = plt.hist(np.random.triangular(0, 5, 9, 1000), bins=100, linewidth=0)
plt.show()
`

When executed via %run, we get the following result:

In [ ]:
%run triangle_hist.py

Note: the above cell needs to be executed twice in order to see the graph

If you wish to pass arguments to a script, simply list them explicitly after the filename %run my_file.py 0 "Hello, World!" or using variables %run $filename {arg0} {arg1}. Additionally, use the -p option to run the code through the Python profiler.

## Scripted Execution

Although the foremost power of Jupyter Notebooks emanates from their interactive flow, it is also possible to run notebooks in a non-interactive mode. Executing notebooks from scripts or the command line provides a powerful way to produce automated reports or similar documents.

Jupyter offers a command line tool that can be used, in its simplest form, for file conversion and execution. As you are probably aware, notebooks can be converted to a number of formats, available from the UI under:

`“File > Download As”`

including HTML, PDF, Python script, and even LaTeX. This functionality is exposed on the command line through an API called **nbconvert**. It is also possible to execute notebooks within Python scripts, but this is already well documented and the examples below should be equally applicable.

It’s important to stress, similarly to `%run`, that while the ability to execute notebooks standalone makes it possible to write all manor of projects entirely within Jupyter notebooks, this is no substitute for breaking up code into standard Python modules and scripts as appropriate.

## On the Command Line

It will become clear later how `nbconvert` empowers developers to create their own automated reporting pipelines, but first let’s look at some simple examples. The basic syntax is:

`
!jupyter nbconvert --to <format> <notebook-filename>
`

For example, to create a PDF, simply write:

`
!jupyter nbconvert --to pdf 01.ipynb
`

This will take the currently saved static content of `01.ipynb` and create a new file called `01.pdf`.

One caveat here is that to convert to PDF requires that you have pandoc (which comes with Anaconda) and LaTeX (which doesn’t) installed. Installation instructions depend on your operating system.

For more information regarding LaTeX, please visit https://nbconvert.readthedocs.io/en/latest/install.html#installing-tex.

By default, `nbconvert` doesn’t execute your notebook code cells. But if you also wish to, you can specify the 
`--execute` flag:

`
jupyter nbconvert --to pdf --execute 01.ipynb
`

A common snag arises from the fact that any error encountered running your notebook will halt execution. Fortunately, you can throw in the `--allow-errors` flag to instruct `nbconvert` to output the error message into the cell output instead.

`
jupyter nbconvert --to pdf --execute --allow-errors 01.ipynb
`

## Parameterization with Environment Variables

Scripted execution is particularly useful for notebooks that don’t always produce the same output, such as if you are processing data that change over time, either from files on disk or pulled down via an API. The resulting documents can easily be emailed to a list of subscribers or uploaded to Amazon S3 for users to download from your website, for example.

In such cases, it’s quite likely you may wish to parameterize your notebooks in order to run them with different initial values. The simplest way to achieve this is using environment variables, which you define before executing the notebook.

Let’s say we want to generate several reports for different dates; in the first cell of our notebook, we can pull this information from an environment variable, which we will name `REPORT_DATE`. The `%env` line magic makes it easy to assign the value of an environment variable to a Python variable.

In the current directory, there's a file named `report.ipynb` which contains a cell with the following code:

`
report_date = %env REPORT_DATE
`

When we execute the following:

`
REPORT_DATE=2018-01-01 jupyter nbconvert --to html --execute report.ipynb
`

A file will be generated called `report.html`, which will contain the successfully executed code.

As all environment variables are strings, we will have to parse them to get the data types we want. For example:

`
A_STRING="Hello, Tim!"
AN_INT=42
A_FLOAT=3.14
A_DATE=2017-12-31 jupyter nbconvert --to html --execute example.ipynb
`

And we simply parse like so:

`
import datetime as dt
the_str = %env A_STRING
int_str = %env AN_INT
my_int = int(int_str)
float_str = %env A_FLOAT
my_float = float(float_str)
date_str = %env A_DATE
my_date = dt.datetime.strptime(date_str, '%Y-%m-%d')
`

Parsing dates is definitely less intuitive than other common data types, but as usual there are several options in Python.

## On Windows

If you’d like to set your environment variables and run your notebook in a single line on Windows, it isn’t quite as simple:

`
cmd /C "set A_STRING=Hello, Tim!&& set AN_INT=42 && set A_FLOAT=3.14 && set A_DATE=2017-12-31&& jupyter nbconvert --to html --execute report.ipynb"
`

Keen readers will notice the lack of a space after defining `A_STRING` and `A_DATE` above. This is because trailing spaces are significant to the Windows `set` command, so while Python will successfully parse the integer and the float by first stripping whitespace, we have to be more careful with our strings.

## Parameterization with Papermill

Using environment variables is fine for simple use-cases, but for anything more complex there are libraries that will let you pass parameters to your notebooks and execute them. With over 1000 stars on GitHub, probably the most popular is Papermill, which can be installed with:

`
pip install papermill.
`

Papermill injects a new cell into your notebook that instantiates the parameters you pass in, parsing numeric inputs for you. This means you can just use the variables without any extra set-up (though dates still need to be parsed). Optionally, you can create a cell in your notebook that defines your default parameter values by clicking “View > Cell Toolbar > Tags” and adding a “parameters” tag to the cell of your choice.

Our earlier example that produced an HTML document now becomes:

`
papermill example.ipynb example-parameterised.ipynb -p my_string "Hello, Tim!" -p my_int 3 -p my_float 3.1416 -p a_date 2017-12-31
jupyter nbconvert example-parameterised.ipynb --to html --output example.html
`

We specify each parameter with the `-p` option and use an intermediary notebook so as not to change the original. It is perfectly possible to overwrite the original `example.ipynb` file, but remember that Papermill will inject a parameter cell.

Now our notebook set-up is much simpler:

`
#my_string, my_int, and my_float are already defined!
import datetime as dt
my_date = dt.datetime.strptime(a_date, '%Y-%m-%d')
`

Our brief glance so far uncovers only the tip of the Papermill iceberg. The library can also execute and collect metrics across notebooks, summarise collections of notebooks, and it provides an API for storing data and Matplotlib plots for access in other scripts or notebooks. It’s all well documented in the GitHub readme, so there’s no need to reiterate here.

It should now be clear that, using this technique, it is possible to write shell or Python scripts that can batch produce multiple documents and be scheduled via tools like crontab to run automatically on a schedule. Powerful stuff!

## Styling Notebooks

If you’re looking for a particular look-and-feel in your notebooks, you can create an external CSS file and load it with Python:

In [ ]:
from IPython.display import HTML
HTML('<style>{}</style>'.format(open('custom.css').read()))

This works because IPython’s HTML objects are inserted directly into the cell output div as raw HTML. In fact, this is equivalent to writing an HTML cell:

`
<style>.css-example { color: darkcyan; }</style>
`

To demonstrate that this works let’s use another HTML cell:

In [ ]:
%%html
<span class='css-example'>This text has a nice colour</span>

Using HTML cells would be fine for one or two lines, but it will typically be cleaner to load an external file as we first saw.

If you would rather customise all your notebooks at once, you can write CSS straight into the `~/.jupyter/custom/custom.css` file in your Jupyter config directory instead, though this will only work when running or converting notebooks on your own computer.

Indeed, all of the aforementioned techniques will also work in notebooks converted to HTML, but will not work in converted PDFs.

To explore your styling options, remember that as Jupyter is just a web app you can use your browser’s dev tools to inspect it while it’s running or delve into some exported HTML output. You will quickly find that it is well-structured: all cells are designated with the `cell` class, text and code cells are likewise respectively demarked with `text_cell` and `code_cell`, inputs and outputs are indicated with input and output, and so on.

There are also various different popular pre-designed themes for Jupyter Notebooks distributed on GitHub. The most popular is jupyterthemes, which is available via:

`pip install jupyterthemes`

and it’s as simple as running:

`jt -t monokai`

to set the “monokai” theme.

If you’re looking to theme JupyterLab instead, there is a growing list of options popping up on GitHub too.